In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from iqa_interpolator import IQA_Interpolator 

In [ ]:
reports = []
for i in range(1,9):
    reports.append(pd.read_excel(f'relatorios/relatorio{i}.xlsx', decimal=','))
df = pd.concat(reports)
df['Data Coleta'] = pd.to_datetime(df['Data Coleta'], format='%d/%m/%Y')
df.head()
# df.to_excel("FinalReport.xlsx")

In [ ]:
print(df.columns)
# print(list(df.Parametro.value_counts().index))

In [ ]:
droppers = ['Cod_Interaguas', 'Tipo Rede', 'Código Ponto', 'Status Ponto', 'Hora Coleta', 'UGRHI', 'Município', 'UF', 'CLASSE', 'Sistema Hídrico', 'Tipo de Sistema Hídrico', 'Localização', 'Captação', 'Altitude', 'Latitude', 'Longitude', 'Inicio Operação', 'Fim Operação']
parametrosRelevantes = ['pH', 'Temperatura da Água', 'Sólido Dissolvido Total', 'Oxigênio Dissolvido', 'Turbidez', 'Nitrogênio-Nitrito', 'Nitrogênio-Nitrato', 'Nitrogênio Kjeldahl', 'Fósforo Total', 'DBO (5, 20)', 'Coliformes Termotolerantes', 'Temperatura do Ar', 'Unidade']
dfFiltered = df.drop(droppers, axis=1)
dfParameters = dfFiltered[dfFiltered['Parametro'].isin(parametrosRelevantes)].reset_index() 
dfParameters.sort_values(by='Data Coleta', inplace=True)

In [ ]:
parametersDfs={}

for parameter in parametrosRelevantes:
    if parameter != 'Coloração' and parameter != 'Unidade':
        parametersDfs[parameter] = dfParameters[dfParameters['Parametro']==parameter].reset_index()
        parametersDfs[parameter]['Valor'] = pd.to_numeric(parametersDfs[parameter]['Valor']) 

unidades = {}

for parameter in parametrosRelevantes:
    if parameter != 'Unidade':
        if parameter not in unidades.keys():
            unidades[parameter] = dfParameters[dfParameters['Parametro']==parameter].reset_index()['Unidade'][0]
print(unidades)
parametrosRelevantes.remove('Unidade')

In [ ]:
lista = list(dfParameters['Data Coleta'])
listaUnica = []
for data in lista:
    if data not in listaUnica:
        listaUnica.append(data)

In [ ]:
Interpolator = IQA_Interpolator()
IQA_Dictionary = {}
coletas=[]
for data in listaUnica:
    coleta = dfParameters[dfParameters['Data Coleta'] == data]
    for key in parametrosRelevantes:
        if key in list(coleta['Parametro']):
            IQA_Dictionary[key] = float(pd.to_numeric(coleta[coleta['Parametro'] == key]['Valor']))
    coletas.append([data, IQA_Dictionary.copy()])
coletas = np.array(coletas)


In [ ]:
df_parametrosNoTempo = pd.DataFrame(coletas)
for k in coletas[0][1].keys():
    lista_k = []
    for i in df_parametrosNoTempo[1]:
        lista_k.append(i[k])
    df_parametrosNoTempo[k] = lista_k
df_parametrosNoTempo.drop(labels=1, axis=1, inplace=True)
df_parametrosNoTempo.rename({0:'Data'},axis=1,inplace=True)
df_parametrosNoTempo['CapacidadeO2'] = df_parametrosNoTempo['Temperatura da Água'].apply(Interpolator.getMaxSolubility)
df_parametrosNoTempo['pctO2'] = df_parametrosNoTempo['Oxigênio Dissolvido']/df_parametrosNoTempo['CapacidadeO2']*100
df_parametrosNoTempo['Nitrogênio Total'] = df_parametrosNoTempo['Nitrogênio Kjeldahl'] + df_parametrosNoTempo['Nitrogênio-Nitrato'] + df_parametrosNoTempo['Nitrogênio-Nitrito']
df_parametrosNoTempo['Temperatura'] = df_parametrosNoTempo['Temperatura da Água'] - df_parametrosNoTempo['Temperatura do Ar'] 
df_parametrosNoTempo.drop(labels=['Nitrogênio-Nitrito', 'Nitrogênio-Nitrato', 'Nitrogênio Kjeldahl', 'Temperatura do Ar', 'Temperatura da Água', 'CapacidadeO2', 'Oxigênio Dissolvido'], axis=1, inplace=True)
df_parametrosNoTempo = df_parametrosNoTempo.set_index("Data")
df_parametrosNoTempo

In [ ]:
print(df_parametrosNoTempo.columns)
print(unidades)

In [ ]:
newUnidades = {}
for k in df_parametrosNoTempo.columns:
    if k in unidades.keys():
        newUnidades[k] = unidades[k]
    else:
        print(k)
newUnidades['pctO2'] = '%'
newUnidades['Nitrogênio Total'] = 'mg/L'
newUnidades['Temperatura'] = 'ºC'


In [ ]:
titulos = {k:k for k in df_parametrosNoTempo.columns}
titulos['pctO2'] = 'Concentração de Oxigênio'
titulos['Temperatura'] = 'Diferença de Temperatura'
print(titulos)

In [ ]:

i = 0
for parameter in df_parametrosNoTempo.columns:
    serieValor = df_parametrosNoTempo[parameter].ewm(span=12, adjust=False).mean()
    serieValorRaw = df_parametrosNoTempo[parameter]
    plt.figure(figsize=(10,10))
    plt.grid(True)
    plt.plot(df_parametrosNoTempo.index, serieValor, c='#00035b', label = 'Filtered')
    plt.plot(df_parametrosNoTempo.index, serieValorRaw, c='xkcd:darkgreen', alpha=0.5, label = 'Raw')
    plt.title(titulos[parameter])
    plt.xlabel("Data de Coleta")
    plt.ylabel(newUnidades[parameter])
    plt.legend()
    if parameter == 'pH':
        plt.ylim((0, 12))
    plt.savefig(f'plots/{i}.png')
    i += 1

In [ ]:
print(df_parametrosNoTempo.index)

In [ ]:
IQATimeline = {}
for row in df_parametrosNoTempo.iterrows():
    IQATimeline[row[0]] = Interpolator.IQA_Result({a:b for a,b in zip(row[1].index,row[1])})

In [ ]:
IQADf = pd.DataFrame(list(IQATimeline.items()),columns = ['Data','IQA'])
serieValor = IQADf['IQA'].ewm(span=12, adjust=False).mean()
serieValorRaw = IQADf['IQA']
plt.figure(figsize=(10,10))
plt.grid(True)
plt.plot(IQADf['Data'], serieValor, c='#00035b')
plt.plot(IQADf['Data'], serieValorRaw, c='xkcd:darkgreen', alpha=0.5)
plt.title("Indice de Qualidade das Àguas")
plt.xlabel("Data de Coleta")
plt.ylabel("Indice de Qualidade das Àguas")
plt.savefig("plots/iqa.png")